# 1. Import and Install Dependencies

In [ ]:
# !pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import ipywidgets as widgets

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [7]:
def create_folder(path, folder_names, no_sequences, start_folder=1):
    for fold in folder_names: 
        for sequence in range(no_sequences):
            try: 
                os.makedirs(os.path.join(path, fold, str(start_folder+sequence)))
            except:
                pass

# 5. Collect Keypoint Values for Training and Testing

In [8]:
def collect_data(press_start=False, countdown = 0):
    cap = cv2.VideoCapture(0)
    
    # Waiting user press start
    while not press_start:
        ret, frame = cap.read()
        cv2.putText(frame, 'PRESS S TO START', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', frame)
                
        if cv2.waitKey(10) & 0xFF == ord('s'):
            press_start=True
            countdown = 5
        continue
    
    # Countdown start
    while countdown > 0:
        ret, frame = cap.read()
        cv2.putText(frame, f'START IN {countdown}', (120,200), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
        countdown -= 1
        cv2.imshow('OpenCV Feed', frame)
        cv2.waitKey(900)                
    
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        # NEW LOOP
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            
            # write video file
            out = cv2.VideoWriter(f'{DATA_PATH}\\{action}\\{sequence}.avi',fourcc, 
                                  fps, (frame_width, frame_height))

            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()
                
                # write video
                out.write(frame)

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
#                 draw_styled_landmarks(image, results)

                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(200)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()
        cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# 6. Action!

Follow
first cell,
1. set directory `DATA_PATH`to save data  
2. set number of videos `no_sequences` to be collect (agreement: 10)
3. set lenght of video `sequence_length` (agreement: 30)
4. set start number of folder name

next cell,  
5. run fuction `create_folder()` to setup folders for follection

next cell,  
6. select action

last cell,  
7. run fuction `collect_data()` to open camera and collect data
8. press 's' to start
9. action after see 'STARTING COLLECTION' for each sequences
10. after finishing one action, go back to 6th step

In [9]:
# Actions that we try to detect
actions = np.array(['หิว', 'ยิ้ม','โกรธ','ร้องไห้','เสียใจ', 
                    'สนุก', 'ขอโทษ', 'ขอบคุณ','แฟน', 'เพื่อน', 
                    'คน', 'คนชรา','เด็ก', 'ทารก', 'ผู้ชาย', 
                    'ผู้หญิง','ผู้ใหญ่', 'วัยรุ่น', 'หนุ่มสาว', 'น้อง', 'ปกติ'
                   ])

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('E:\Knowledge\_BADs_at_NIDA\BADS7203_Image and video analytics\_Project\SLR\dataset\jo')

# Thirty videos worth of data
no_sequences = 10

# Videos are going to be 30 frames in length
sequence_length = 30

# # Folder start
start_folder = 1

# setup veido writer
video = cv2.VideoCapture(0)
frame_width = int(video.get(3))
frame_height = int(video.get(4))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 10

In [ ]:
# create folders for follection
create_folder(DATA_PATH, actions, no_sequences, start_folder)

In [ ]:
action_select = widgets.Dropdown(
                options=actions,
                value=actions[0],
                description='Number:',
                disabled=False,
                )
action_select

In [11]:
# action = action_select.value
action = 'ปกติ'
print('ปกติ')
collect_data()

ปกติ
